In [1]:
import pandas as pd
import os

lista_imagens = list(i for i in os.listdir(r'samples') if i.endswith('.png'))

df = pd.DataFrame({
    'imagens': lista_imagens
})
df

,imagens
0,226md.png
1,22d5n.png
2,2356g.png
3,23mdg.png
4,23n88.png
...,...
1035,yx2d4.png
1036,yxd7m.png
1037,yy824.png
1038,yyg5g.png


In [2]:
df['solucao'] = df['imagens'].apply(lambda x: os.path.splitext(x)[0])
df

,imagens,solucao
0,226md.png,226md
1,22d5n.png,22d5n
2,2356g.png,2356g
3,23mdg.png,23mdg
4,23n88.png,23n88
...,...,...
1035,yx2d4.png,yx2d4
1036,yxd7m.png,yxd7m
1037,yy824.png,yy824
1038,yyg5g.png,yyg5g


In [3]:
df['caminho_imagem'] = df['imagens'].apply(lambda x: os.path.join('samples',x))
df

,imagens,solucao,caminho_imagem
0,226md.png,226md,samples\226md.png
1,22d5n.png,22d5n,samples\22d5n.png
2,2356g.png,2356g,samples\2356g.png
3,23mdg.png,23mdg,samples\23mdg.png
4,23n88.png,23n88,samples\23n88.png
...,...,...,...
1035,yx2d4.png,yx2d4,samples\yx2d4.png
1036,yxd7m.png,yxd7m,samples\yxd7m.png
1037,yy824.png,yy824,samples\yy824.png
1038,yyg5g.png,yyg5g,samples\yyg5g.png


In [4]:
df['solucao']

0       226md
1       22d5n
2       2356g
3       23mdg
4       23n88
        ...  
1035    yx2d4
1036    yxd7m
1037    yy824
1038    yyg5g
1039    yyn57
Name: solucao, Length: 1040, dtype: object

In [5]:
os.path.splitext('yyn57.png')[0]

'yyn57'

In [6]:
lista_imagens

['226md.png',
 '22d5n.png',
 '2356g.png',
 '23mdg.png',
 '23n88.png',
 '243mm.png',
 '244e2.png',
 '245y5.png',
 '24f6w.png',
 '24pew.png',
 '25257.png',
 '253dc.png',
 '25egp.png',
 '25m6p.png',
 '25p2m.png',
 '25w53.png',
 '264m5.png',
 '268g2.png',
 '28348.png',
 '28x47.png',
 '2b827.png',
 '2bg48.png',
 '2cegf.png',
 '2cg58.png',
 '2cgyx.png',
 '2en7g.png',
 '2enf4.png',
 '2fxgd.png',
 '2g783.png',
 '2g7nm.png',
 '2gyb6.png',
 '2mg87.png',
 '2mpnn.png',
 '2n73f.png',
 '2nbcx.png',
 '2nf26.png',
 '2npg6.png',
 '2nx38.png',
 '2p2y8.png',
 '2pfpn.png',
 '2w4y7.png',
 '2wc38.png',
 '2wx73.png',
 '2x7bm.png',
 '2xc2n.png',
 '2ycn8.png',
 '2yggg.png',
 '325fb.png',
 '32cnn.png',
 '32dnn.png',
 '33b22.png',
 '33f7m.png',
 '33n73.png',
 '33ng4.png',
 '33p4e.png',
 '34b84.png',
 '34fxm.png',
 '34pcn.png',
 '368y5.png',
 '36bc2.png',
 '36nx4.png',
 '36w25.png',
 '373gb.png',
 '377xx.png',
 '378e5.png',
 '37d52.png',
 '37ep6.png',
 '387g2.png',
 '38n57.png',
 '3b4we.png',
 '3bd8f.png',
 '3bfn

In [ ]:
# Importações necessárias
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from PIL import Image



altura = 100
largura = 100


In [ ]:
# Função para carregar e pré-processar as imagens
def load_images(file_paths):
    images = []
    for path in file_paths:
        image = Image.open(path).convert('RGB')
        image = image.resize((altura, largura))
        image = np.array(image) / 255.0
        images.append(image)
    return np.array(images)

In [ ]:
X = load_images(df['caminho_imagem'])

y = df['solucao'].apply(list)

characters = set(char for sublist in y for char in sublist)
num_classes = len(characters)

char_to_index = {char: i for i, char in enumerate(characters)}
index_to_char = {i: char for char, i in char_to_index.items()}

y = [[char_to_index[char] for char in sublist] for sublist in y]

max_length = max(len(sublist) for sublist in y)
y = tf.keras.preprocessing.sequence.pad_sequences(y, maxlen=max_length, padding='post')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:


# Definição do modelo seq2seq
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(altura, largura, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.RepeatVector(max_length),
    layers.LSTM(64, return_sequences=True),
    # layers.TimeDistributed(layers.Dense(num_classes, activation='softmax'))  # Camada densa para previsão de cada caractere
    layers.Dense(num_classes, activation='softmax')  # Atualização da função de ativação para softmax
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 868ms/step - accuracy: 0.0977 - loss: 2.9774 - val_accuracy: 0.0971 - val_loss: 2.9273
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 595ms/step - accuracy: 0.0973 - loss: 2.9166 - val_accuracy: 0.0971 - val_loss: 2.8977
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 620ms/step - accuracy: 0.1198 - loss: 2.8355 - val_accuracy: 0.1019 - val_loss: 2.8266
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 584ms/step - accuracy: 0.1610 - loss: 2.7384 - val_accuracy: 0.1519 - val_loss: 2.7245
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 621ms/step - accuracy: 0.2180 - loss: 2.5741 - val_accuracy: 0.1663 - val_loss: 2.6873
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 612ms/step - accuracy: 0.2370 - loss: 2.4566 - val_accuracy: 0.2038 - val_loss: 2.5531
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 608ms/step - accuracy: 0.2698 - loss: 2.3087 - val_accuracy: 0.2106 - val_loss: 2.4751
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 625ms/step - accuracy: 0.2892 - loss: 2.1531 - val_accuracy: 0.2163 - 

In [12]:
def predict_captcha(model, captcha_image_path, char_to_index, index_to_char):
    captcha_image = Image.open(captcha_image_path).convert('RGB')
    captcha_image = captcha_image.resize((altura, largura))
    captcha_image = np.array(captcha_image) / 255.0
    captcha_image = np.expand_dims(captcha_image, axis=0) 
    
    predictions = model.predict(captcha_image)
    
    decoded_predictions = []
    for prediction in predictions[0]:
        predicted_index = np.argmax(prediction)
        predicted_char = index_to_char[predicted_index]
        decoded_predictions.append(predicted_char)
    
    return ''.join(decoded_predictions)


In [13]:
captcha_image_path = r'samples\ywn6f.png'  
predicted_solution = predict_captcha(model, captcha_image_path, char_to_index, index_to_char)
print('Predicted solution:', predicted_solution)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
Predicted solution: ywn6f


In [15]:
captcha_image_path = r'samples\3fbxd.png'  
predicted_solution = predict_captcha(model, captcha_image_path, char_to_index, index_to_char)
print('Predicted solution:', predicted_solution)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted solution: 3fbxd


In [17]:
model.save('epochs_100_seq2seq.keras')

In [18]:
from tensorflow.keras.models import load_model

# Carregar o modelo
modelo_carregado = load_model('epochs_100_seq2seq.keras')


e:\Programação\ml_captcha_image\venv\lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 13 variables whereas the saved optimizer has 24 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [20]:
import pickle

with open('char_to_index.pickle', 'wb') as handle:
    pickle.dump(char_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('index_to_char.pickle', 'wb') as handle:
    pickle.dump(index_to_char, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [21]:
import pickle

with open('char_to_index.pickle', 'rb') as handle:
    char_to_index_load = pickle.load(handle)

with open('index_to_char.pickle', 'rb') as handle:
    index_to_char_load = pickle.load(handle)